# Implementing gridsearchcv for XGBoost

In [43]:
import pandas as pd
import numpy as np
import modelinghelper as helper
import xgboost as xgb
import os
# supress future warnings
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [10]:
#set up random seed
rng = 42

# set up folder to save results
output_path = 'gridsearch_all_models'
dataset = 'export'
mod_name = "XGBoost"
run_num = "run_1"

if not os.path.exists(output_path):
    os.makedirs(output_path)
else:
    print("Folder already exists")

# file name
prefix = f'{output_path}/{dataset}/{mod_name}{run_num}'

Folder already exists


In [11]:
# read in data
p = '../data/'
input_name = 'cleaned.csv'

df = pd.read_csv(f'{p}{input_name}')
# convert month to string
df['ship_date_mm'] = df['ship_date_mm'].astype(str)

In [12]:
non_feature_cols = ['control_number', 'disp_date', 'i_e', 'ship_date',
                    'cartons', 'qty', 'unit', 'specific_generic_name',
                    'genus', 'species', 'wildlf_cat',
                    'disp_date_yyyy', 'disp_date_mm', 'disp_ship_date']

target = ['act', 'dp_cd']

feature_cols = ['species_code', 'wildlf_desc', 'wildlf_cat',
                'ctry_org', 'ctry_ie','purp', 'src', 'trans_mode', 'pt_cd',
                'value', 'ship_date_mm']

export_df = helper.df_filtering(df, i_e = 'E', f_cols = feature_cols)

In [13]:
X_train, X_test, y_train, y_test = helper.data_split(export_df)

### XGBoost

In [75]:
clf_xgb = xgb.XGBClassifier(random_state = rng, verbosity = 0, silent=True, objective='binary:logistic', use_label_encoder=False)

xgb_params = {
    'clf__n_estimators' : [100, 500, 1000],
    "clf__max_depth" : [1,2,3,4,5,6,7,8,9,10],
    "clf__learning_rate" : np.linspace(.03, .5, 20),
    "clf__subsample": [.25,.5,.75],
    "clf__colsample_bylevel" : [.5,.6,.7],

     #early stopping?
}

xgb_pipe = helper.gridsearch_pipeline(X_train, y_train, clf_xgb, xgb_params)

Fitting 5 folds for each of 5400 candidates, totalling 27000 fits
[CV 1/5] END clf__colsample_bylevel=0.5, clf__learning_rate=0.03, clf__max_depth=1, clf__n_estimators=100, clf__subsample=0.25;, score=0.034 total time=   3.0s
[CV 2/5] END clf__colsample_bylevel=0.5, clf__learning_rate=0.03, clf__max_depth=1, clf__n_estimators=100, clf__subsample=0.25;, score=0.057 total time=   2.8s
[CV 3/5] END clf__colsample_bylevel=0.5, clf__learning_rate=0.03, clf__max_depth=1, clf__n_estimators=100, clf__subsample=0.25;, score=0.062 total time=   2.7s
[CV 4/5] END clf__colsample_bylevel=0.5, clf__learning_rate=0.03, clf__max_depth=1, clf__n_estimators=100, clf__subsample=0.25;, score=0.057 total time=   2.8s
[CV 5/5] END clf__colsample_bylevel=0.5, clf__learning_rate=0.03, clf__max_depth=1, clf__n_estimators=100, clf__subsample=0.25;, score=0.057 total time=   2.7s
[CV 1/5] END clf__colsample_bylevel=0.5, clf__learning_rate=0.03, clf__max_depth=1, clf__n_estimators=100, clf__subsample=0.5;, score=

KeyboardInterrupt: 